**Import Necessary Libraries **

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report


Load and Preprocess the Data


In [ ]:
# Load the dataset
df = pd.read_csv('/content/heart_disease_data (1).csv')

# Split features and target
X = df.drop('target', axis=1)
y = df['target']
from keras.models import load_model




# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


Build the Simple Neural Network Model

In [ ]:
def build_simple_model(input_shape):
    model = models.Sequential([
        layers.Dense(32, activation='relu', input_shape=(input_shape,)),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

simple_model = build_simple_model(X_train.shape[1])


Train the Simple Model

In [ ]:
history = simple_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), verbose=2)


Epoch 1/100
6/6 - 1s - loss: 0.6866 - accuracy: 0.5691 - val_loss: 0.6472 - val_accuracy: 0.5902 - 1s/epoch - 187ms/step
Epoch 2/100
6/6 - 0s - loss: 0.6446 - accuracy: 0.6409 - val_loss: 0.6197 - val_accuracy: 0.6230 - 69ms/epoch - 12ms/step
Epoch 3/100
6/6 - 0s - loss: 0.6129 - accuracy: 0.6740 - val_loss: 0.5963 - val_accuracy: 0.6557 - 72ms/epoch - 12ms/step
Epoch 4/100
6/6 - 0s - loss: 0.5838 - accuracy: 0.7017 - val_loss: 0.5764 - val_accuracy: 0.6721 - 58ms/epoch - 10ms/step
Epoch 5/100
6/6 - 0s - loss: 0.5578 - accuracy: 0.7127 - val_loss: 0.5588 - val_accuracy: 0.6557 - 76ms/epoch - 13ms/step
Epoch 6/100
6/6 - 0s - loss: 0.5362 - accuracy: 0.7348 - val_loss: 0.5428 - val_accuracy: 0.7049 - 79ms/epoch - 13ms/step
Epoch 7/100
6/6 - 0s - loss: 0.5145 - accuracy: 0.7459 - val_loss: 0.5279 - val_accuracy: 0.7049 - 57ms/epoch - 9ms/step
Epoch 8/100
6/6 - 0s - loss: 0.4942 - accuracy: 0.7790 - val_loss: 0.5149 - val_accuracy: 0.6885 - 60ms/epoch - 10ms/step
Epoch 9/100
6/6 - 0s - los

Build and Train the Optimized Model

In [ ]:
def build_optimized_model(input_shape):
    model = models.Sequential([
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(input_shape,)),
        layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

optimized_model = build_optimized_model(X_train.shape[1])

# Apply Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history_opt = optimized_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=2)


Epoch 1/100
6/6 - 1s - loss: 1.0279 - accuracy: 0.6851 - val_loss: 0.9765 - val_accuracy: 0.7705 - 965ms/epoch - 161ms/step
Epoch 2/100
6/6 - 0s - loss: 0.9895 - accuracy: 0.7127 - val_loss: 0.9483 - val_accuracy: 0.8033 - 43ms/epoch - 7ms/step
Epoch 3/100
6/6 - 0s - loss: 0.9584 - accuracy: 0.7569 - val_loss: 0.9222 - val_accuracy: 0.7869 - 44ms/epoch - 7ms/step
Epoch 4/100
6/6 - 0s - loss: 0.9282 - accuracy: 0.7956 - val_loss: 0.8994 - val_accuracy: 0.7869 - 58ms/epoch - 10ms/step
Epoch 5/100
6/6 - 0s - loss: 0.9008 - accuracy: 0.8122 - val_loss: 0.8779 - val_accuracy: 0.8033 - 42ms/epoch - 7ms/step
Epoch 6/100
6/6 - 0s - loss: 0.8748 - accuracy: 0.8343 - val_loss: 0.8576 - val_accuracy: 0.8033 - 59ms/epoch - 10ms/step
Epoch 7/100
6/6 - 0s - loss: 0.8510 - accuracy: 0.8398 - val_loss: 0.8386 - val_accuracy: 0.8197 - 60ms/epoch - 10ms/step
Epoch 8/100
6/6 - 0s - loss: 0.8275 - accuracy: 0.8453 - val_loss: 0.8211 - val_accuracy: 0.8197 - 61ms/epoch - 10ms/step
Epoch 9/100
6/6 - 0s - lo

Evaluate the Model

In [ ]:
# Simple Model Evaluation
simple_model.evaluate(X_test, y_test)

# Optimized Model Evaluation
optimized_model.evaluate(X_test, y_test)

# Confusion Matrix and Classification Report for the Optimized Model
y_pred = (optimized_model.predict(X_test) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


2/2 [==============================] - 0s 6ms/step
[[25  4]
 [ 8 24]]
              precision    recall  f1-score   support

           0       0.76      0.86      0.81        29
           1       0.86      0.75      0.80        32

    accuracy                           0.80        61
   macro avg       0.81      0.81      0.80        61
weighted avg       0.81      0.80      0.80        61

